In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals, annotations

import logging
import os
import pickle
import time
import numpy as np
import argparse
import json
import gzip
from pathlib import Path
import psutil
import sys
import time

# ----------------- Spark 相关导入 -----------------
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, rand, monotonically_increasing_id, input_file_name, regexp_extract
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, ArrayType, DoubleType, LongType, BooleanType

# ----------------- 自定义工具导入 -----------------
# 请确保 utils、ml.utils 模块在 Python 环境中可用
from utils import load_config
from ml.utils import load_application_classification_cnn_model, load_traffic_classification_cnn_model, normalise_cm

# ----------------- PyTorch 相关导入 -----------------
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# ----------------- ART、聚类、PCA 和可视化相关导入 -----------------
from art.estimators.classification import PyTorchClassifier
from art.defences.detector.poison import ClusteringAnalyzer
from art.defences.detector.poison.poison_filtering_defence import PoisonFilteringDefence
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import matplotlib.cm as cm

# # 配置设备
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 直接指定使用 CPU 设备
device = torch.device("cpu")


In [2]:
class_num=15
labels = [str(i) for i in range(0, class_num)]

In [3]:
from torch.utils.data import DataLoader
from ml.dataset import dataset_collate_function
import datasets
import multiprocessing
def train_dataloader(parquet_path):
    # expect to get train folder
    #print("self.hparams.data_path",self.hparams.data_path)
    # data_path=self.hparams.data_path
    # df = pd.read_parquet(data_path)
    # print(df.head())
    #dataset_dict = datasets.load_dataset(self.hparams.data_path)
    dataset_dict = datasets.load_dataset(parquet_path, keep_in_memory=False)
    dataset = dataset_dict[list(dataset_dict.keys())[0]]
    try:
        num_workers = multiprocessing.cpu_count()
    except:
        num_workers = 1
    dataloader = DataLoader(
        dataset,
        #batch_size=4,
        batch_size=128,
        num_workers=num_workers,
        collate_fn=dataset_collate_function,
        shuffle=True,
    )
    print("num_workers:",num_workers)
    return dataloader

parquet_path = "/home/zmx/Activation_Clustering-3/dirty/poison_trigger4_30_254_65534_20_0.5_0.1/poisoned_dataset/application_classification/train.parquet"
dataloader=train_dataloader(parquet_path)

Resolving data files:   0%|          | 0/105 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/78 [00:00<?, ?it/s]

num_workers: 52


In [4]:
model_path = "/home/zmx/Activation_Clustering-3/dirty/model_poison_trigger4_30_254_65534_20_0.5_0.1/application_classification.cnn.model"
from ml.utils import load_application_classification_cnn_model
model = load_application_classification_cnn_model(model_path, gpu=True)
model.eval()  # 设为评估模式

/home/zmx/.conda/envs/deep-packet/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


CNN(
  (conv1): Sequential(
    (0): Conv1d(1, 200, kernel_size=(4,), stride=(3,))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv1d(200, 200, kernel_size=(5,), stride=(1,))
    (1): ReLU()
  )
  (max_pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=49400, out_features=200, bias=True)
    (1): Dropout(p=0.05, inplace=False)
    (2): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): Dropout(p=0.05, inplace=False)
    (2): ReLU()
  )
  (fc3): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): Dropout(p=0.05, inplace=False)
    (2): ReLU()
  )
  (out): Linear(in_features=50, out_features=15, bias=True)
)

In [5]:
# 定义损失函数和优化器（此处仅用于 ART 分类器的构建）
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:
class_num = 15
input_dimension = 1500  # 根据模型实际输入尺寸设置

classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_dimension,),  # 例如信号长度为1500
    nb_classes=class_num
)

In [7]:
# 修改数据收集部分
all_features = []
all_labels = []
all_is_poisoned = []

for batch in dataloader:
    # 获取字典中各字段，并转移到模型所在设备（如 GPU）
    x_batch = batch["feature"].float().to(model.device)
    y_batch = batch["label"].long().to(model.device)
    poisoned_batch = batch["is_poisoned"].bool().to(model.device)
    
    # 立即转移到CPU并释放GPU内存
    all_features.append(x_batch.cpu().numpy())
    all_labels.append(y_batch.cpu().numpy())
    all_is_poisoned.append(poisoned_batch.cpu().numpy())
    
    # 清理GPU内存
    del x_batch, y_batch, poisoned_batch
    torch.cuda.empty_cache()

# 最后再合并
all_features = np.concatenate(all_features, axis=0)
all_labels = np.concatenate(all_labels, axis=0)
all_is_poisoned = np.concatenate(all_is_poisoned, axis=0)

In [8]:
torch.cuda.empty_cache()

In [9]:
from art.defences.detector.poison import ActivationDefence
# 设置批次大小
BATCH_SIZE = 1000  # 可以根据GPU内存调整

# 初始化结果列表
all_activations = []
all_labels_batch = []
all_is_poisoned_batch = []

# 分批处理数据
for i in range(0, len(all_features), BATCH_SIZE):
    # 获取当前批次
    batch_features = all_features[i:i+BATCH_SIZE]
    batch_labels = all_labels[i:i+BATCH_SIZE]
    batch_is_poisoned = all_is_poisoned[i:i+BATCH_SIZE]
    
    # 创建当前批次的defence
    defence_batch = ActivationDefence(
        classifier=classifier,
        x_train=batch_features,
        y_train=batch_labels,
        generator=None,
        ex_re_threshold=None
    )
    
    # 获取当前批次的激活
    try:
        # 获取激活
        activations = defence_batch._get_activations()
        
        # 将激活转移到CPU并存储
        if isinstance(activations, np.ndarray):
            all_activations.append(activations)
        else:
            all_activations.append(activations.cpu().numpy())
            
        # 存储标签和投毒信息
        all_labels_batch.append(batch_labels)
        all_is_poisoned_batch.append(batch_is_poisoned)
        
        # 清理GPU内存
        del defence_batch
        torch.cuda.empty_cache()
        
        print(f"处理批次 {i//BATCH_SIZE + 1}/{len(all_features)//BATCH_SIZE + 1}")
        
    except Exception as e:
        print(f"处理批次 {i//BATCH_SIZE + 1} 时出错: {str(e)}")
        continue

# 合并所有批次的结果
all_activations = np.concatenate(all_activations, axis=0)
all_labels_batch = np.concatenate(all_labels_batch, axis=0)
all_is_poisoned_batch = np.concatenate(all_is_poisoned_batch, axis=0)

torch.cuda.empty_cache()

处理批次 1/6578
处理批次 2/6578
处理批次 3/6578
处理批次 4/6578
处理批次 5/6578
处理批次 6/6578
处理批次 7/6578
处理批次 8/6578
处理批次 9/6578
处理批次 10/6578
处理批次 11/6578
处理批次 12/6578
处理批次 13/6578
处理批次 14/6578
处理批次 15/6578
处理批次 16/6578
处理批次 17/6578
处理批次 18/6578
处理批次 19/6578
处理批次 20/6578
处理批次 21/6578
处理批次 22/6578
处理批次 23/6578
处理批次 24/6578
处理批次 25/6578
处理批次 26/6578
处理批次 27/6578
处理批次 28/6578
处理批次 29/6578
处理批次 30/6578
处理批次 31/6578
处理批次 32/6578
处理批次 33/6578
处理批次 34/6578
处理批次 35/6578
处理批次 36/6578
处理批次 37/6578
处理批次 38/6578
处理批次 39/6578
处理批次 40/6578
处理批次 41/6578
处理批次 42/6578
处理批次 43/6578
处理批次 44/6578
处理批次 45/6578
处理批次 46/6578
处理批次 47/6578
处理批次 48/6578
处理批次 49/6578
处理批次 50/6578
处理批次 51/6578
处理批次 52/6578
处理批次 53/6578
处理批次 54/6578
处理批次 55/6578
处理批次 56/6578
处理批次 57/6578
处理批次 58/6578
处理批次 59/6578
处理批次 60/6578
处理批次 61/6578
处理批次 62/6578
处理批次 63/6578
处理批次 64/6578
处理批次 65/6578
处理批次 66/6578
处理批次 67/6578
处理批次 68/6578
处理批次 69/6578
处理批次 70/6578
处理批次 71/6578
处理批次 72/6578
处理批次 73/6578
处理批次 74/6578
处理批次 75/6578
处理批次 76/6578
处理批次 77/6578
处理批次 78/

In [10]:
# 1. 创建防御对象并设置FastICA
final_defence = ActivationDefence(
    classifier=classifier,
    x_train=all_features,
    y_train=all_labels,
    generator=None,
    ex_re_threshold=None,
)
# 设置降维方法为FastICA
final_defence.reduce = "PCA"
final_defence.clustering_method = "KMeans"

In [11]:
# 2. 进行防御检测
final_defence.activations_by_class = final_defence._segment_by_class(all_activations, all_labels)
(
    final_defence.clusters_by_class,
    final_defence.red_activations_by_class,
) = final_defence.cluster_activations()

In [12]:
# ----------------- 基于激活聚类的中毒检测 ----------------

# 运行检测（传入特征和对应标签）
report, is_clean_pred_lst = final_defence.detect_poison()
# 转成布尔：True=clean, False=poison
is_clean_pred = np.array(is_clean_pred_lst, dtype=bool)

# all_is_poisoned_batch: True=poisoned, False=clean
# ground‐truth is_clean: True=clean, False=poison
is_clean_gt = ~all_is_poisoned_batch

result_json = final_defence.evaluate_defence(is_clean=is_clean_gt)
import json
print(json.dumps(report, indent=4))

{
    "cluster_analysis": "smaller",
    "suspicious_clusters": 15,
    "Class_0": {
        "cluster_0": {
            "ptc_data_in_cluster": 0.82,
            "suspicious_cluster": false
        },
        "cluster_1": {
            "ptc_data_in_cluster": 0.18,
            "suspicious_cluster": true
        }
    },
    "Class_1": {
        "cluster_0": {
            "ptc_data_in_cluster": 0.87,
            "suspicious_cluster": false
        },
        "cluster_1": {
            "ptc_data_in_cluster": 0.13,
            "suspicious_cluster": true
        }
    },
    "Class_2": {
        "cluster_0": {
            "ptc_data_in_cluster": 0.61,
            "suspicious_cluster": false
        },
        "cluster_1": {
            "ptc_data_in_cluster": 0.39,
            "suspicious_cluster": true
        }
    },
    "Class_3": {
        "cluster_0": {
            "ptc_data_in_cluster": 0.46,
            "suspicious_cluster": true
        },
        "cluster_1": {
            "ptc_data_

In [13]:
# 打印结果
print(f"特征维度: {all_features.shape}")
print(f"标签数量: {len(all_labels)}")
print(f"投毒样本比例: {np.mean(all_is_poisoned):.2%}")
print("检测评估结果：", result_json)

特征维度: (6577628, 1, 1500)
标签数量: 6577628
投毒样本比例: 12.62%
检测评估结果： {"class_0": {"TruePositive": {"rate": -1, "numerator": 0, "denominator": 0}, "TrueNegative": {"rate": 81.81, "numerator": 6333, "denominator": 7741}, "FalsePositive": {"rate": 18.19, "numerator": 1408, "denominator": 7741}, "FalseNegative": {"rate": -1, "numerator": 0, "denominator": 0}}, "class_1": {"TruePositive": {"rate": -1, "numerator": 0, "denominator": 0}, "TrueNegative": {"rate": 87.2, "numerator": 41939, "denominator": 48094}, "FalsePositive": {"rate": 12.8, "numerator": 6155, "denominator": 48094}, "FalseNegative": {"rate": -1, "numerator": 0, "denominator": 0}}, "class_2": {"TruePositive": {"rate": 74.4, "numerator": 617511, "denominator": 830040}, "TrueNegative": {"rate": 91.36, "numerator": 887615, "denominator": 971567}, "FalsePositive": {"rate": 8.64, "numerator": 83952, "denominator": 971567}, "FalseNegative": {"rate": 25.6, "numerator": 212529, "denominator": 830040}}, "class_3": {"TruePositive": {"rate": -1

In [14]:
import json

# 如果 result_json 是字符串，先转换为字典
if isinstance(result_json, str):
    result_json = json.loads(result_json)

class_2_result = result_json["class_2"]

TP_num = class_2_result["TruePositive"]["numerator"]
FN_num = class_2_result["FalseNegative"]["numerator"]
FP_num = class_2_result["FalsePositive"]["numerator"]
TN_num = class_2_result["TrueNegative"]["numerator"]

total_positive = TP_num + FN_num
total_negative = FP_num + TN_num

if total_positive != 0:
    TPR = TP_num / total_positive * 100  # 检出率 (TPR)
    FNR = FN_num / total_positive * 100         # 漏检率 (FNR)
else:
    TPR = None
    FNR = None

if total_negative != 0:
    FPR = FP_num / total_negative * 100   # 误报率 (FPR)
else:
    FPR = None

print("\n检测性能指标：")
if TPR is not None:
    print(f"检出率: {TPR:.2f}%")
else:
    print("检出率: 未定义（无正样本）")

if FPR is not None:
    print(f"误报率：{FPR:.2f}%")
else:
    print("误报率: 未定义（无正样本）")

if FNR is not None:
    print(f"漏检率: {FNR:.2f}%")
else:
    print("漏检率: 未定义（无正样本）")

print(parquet_path)



检测性能指标：
检出率: 74.40%
误报率：8.64%
漏检率: 25.60%
/home/zmx/Activation_Clustering-3/clean/poison_trigger4_30_254_65534_20_0.5_0.1/poisoned_dataset/application_classification/train.parquet


In [15]:
# 初始化计数器和累加器
valid_tp_count = 0  # 有效TruePositive类别数
valid_fn_count = 0  # 有效FalseNegative类别数
total_fp_count = 0  # 所有类别数（用于FalsePositive）

tp_sum = 0  # TruePositive累加
fn_sum = 0  # FalseNegative累加
fp_sum = 0  # FalsePositive累加

# 遍历所有类别
for class_name, metrics in result_json.items():
    # TruePositive
    if metrics["TruePositive"]["rate"] != -1:
        tp_sum += metrics["TruePositive"]["rate"]
        valid_tp_count += 1
    
    # FalseNegative
    if metrics["FalseNegative"]["rate"] != -1:
        fn_sum += metrics["FalseNegative"]["rate"]
        valid_fn_count += 1
    
    # FalsePositive（所有类别都有效）
    fp_sum += metrics["FalsePositive"]["rate"]
    total_fp_count += 1

# 计算平均指标
avg_detection_rate = tp_sum / valid_tp_count if valid_tp_count > 0 else 0
avg_miss_rate = fn_sum / valid_fn_count if valid_fn_count > 0 else 0
avg_false_alarm_rate = fp_sum / total_fp_count

print("\n所有类别的平均检测性能指标：")
print(f"平均检出率: {avg_detection_rate:.2f}%")
print(f"平均误报率: {avg_false_alarm_rate:.2f}%")
print(f"平均漏检率: {avg_miss_rate:.2f}%")

print("\n各类别详细指标：")
for class_name, metrics in result_json.items():
    print(f"\n{class_name}:")
    tp_rate = metrics["TruePositive"]["rate"]
    fn_rate = metrics["FalseNegative"]["rate"]
    fp_rate = metrics["FalsePositive"]["rate"]
    
    print(f"检出率: {tp_rate if tp_rate != -1 else 'N/A'}%")
    print(f"误报率: {fp_rate:.2f}%")
    print(f"漏检率: {fn_rate if fn_rate != -1 else 'N/A'}%")


所有类别的平均检测性能指标：
平均检出率: 74.40%
平均误报率: 29.88%
平均漏检率: 25.60%

各类别详细指标：

class_0:
检出率: N/A%
误报率: 18.19%
漏检率: N/A%

class_1:
检出率: N/A%
误报率: 12.80%
漏检率: N/A%

class_2:
检出率: 74.4%
误报率: 8.64%
漏检率: 25.6%

class_3:
检出率: N/A%
误报率: 45.91%
漏检率: N/A%

class_4:
检出率: N/A%
误报率: 39.74%
漏检率: N/A%

class_5:
检出率: N/A%
误报率: 42.64%
漏检率: N/A%

class_6:
检出率: N/A%
误报率: 12.83%
漏检率: N/A%

class_7:
检出率: N/A%
误报率: 0.84%
漏检率: N/A%

class_8:
检出率: N/A%
误报率: 49.36%
漏检率: N/A%

class_9:
检出率: N/A%
误报率: 33.02%
漏检率: N/A%

class_10:
检出率: N/A%
误报率: 27.45%
漏检率: N/A%

class_11:
检出率: N/A%
误报率: 26.65%
漏检率: N/A%

class_12:
检出率: N/A%
误报率: 48.89%
漏检率: N/A%

class_13:
检出率: N/A%
误报率: 40.73%
漏检率: N/A%

class_14:
检出率: N/A%
误报率: 40.55%
漏检率: N/A%


In [16]:
# 4. 可视化：每个类别随机选取 50 个正常样本并绘制实心方块，投毒样本绘制红色空心三角形
import numpy as np

# 如果 red_acts_dict 是列表
red_acts_list = final_defence.red_activations_by_class  # 其实是一个 list

all_red_acts = []
all_labels_arr = []

# 如果你知道它是 [class_0_acts, class_1_acts, ...] 这种结构，
# 或文档说明第 i 个元素对应第 i 个类别，可以使用枚举
for i, acts in enumerate(red_acts_list):
    # acts 就是第 i 个类别的降维激活
    all_red_acts.append(acts)
    # 如果你能对应上类别标签，假设 i 就是类别标签，或者另有某个映射
    # 这里演示假设 i 即为类别
    all_labels_arr.extend([i]*len(acts))

all_red_acts = np.concatenate(all_red_acts, axis=0)
all_labels_arr = np.array(all_labels_arr)
print(all_labels_arr)


print("合并后的降维激活：", all_red_acts.shape)
print("合并后的标签：", all_labels_arr.shape)

[ 0  0  0 ... 14 14 14]
合并后的降维激活： (6577628, 10)
合并后的标签： (6577628,)


In [17]:
# features_2d=all_red_acts
# all_labels_batch=all_labels_arr

# plt.figure(figsize=(10, 8),dpi=1200)
# unique_labels = np.unique(all_labels_batch)

# # 给不同类别分配颜色
# colors = plt.cm.get_cmap("tab20b", len(unique_labels))
# # colors = plt.cm.get_cmap("plasma", len(unique_labels))
# label_to_color = {lab: colors(i) for i, lab in enumerate(unique_labels)}

# n_per_class = 50

# for lab in unique_labels:
#     # 找到该类别、且未投毒的样本索引
#     idx_clean_mask = np.where((all_labels_batch == lab) & (~all_is_poisoned_batch))[0]
#     if len(idx_clean_mask) == 0:
#         continue
#     # 随机选取 50 个，或不足 50 个则全部选
#     select_num = min(n_per_class, len(idx_clean_mask))
#     selected_idx = np.random.choice(idx_clean_mask, size=select_num, replace=False)
#     if lab != 2:
#         # 绘制散点（实心方块）
#         plt.scatter(
#             features_2d[selected_idx, 0],
#             features_2d[selected_idx, 1],
#             color=label_to_color[lab],
#             marker='s',  # 实心方块
#             s=100,
#             alpha=0.8,
#             # label=f"Other Class (Clean)" if lab not in plt.gca().get_legend_handles_labels()[1] else ""
#             # 为防止重复图例，每类只在首次绘制时加 legend
#         )
#     if lab == 2:
#         # 绘制散点（）
#         plt.scatter(
#             features_2d[selected_idx, 0],
#             features_2d[selected_idx, 1],
#             color="#C82423",
#             marker='.',  # 
#             s=130,
#             alpha=0.8,
#             label=f"Target Class (Clean)" if lab not in plt.gca().get_legend_handles_labels()[1] else ""
#             # 为防止重复图例，每类只在首次绘制时加 legend
#         )
# poison_idx = np.where(all_is_poisoned_batch)[0]

# # 如果投毒样本超过50个，则随机选取50个
# if len(poison_idx) > 50:
#     np.random.shuffle(poison_idx)
#     poison_idx = poison_idx[:50]

# if len(poison_idx) > 0:
#     plt.scatter(
#         features_2d[poison_idx, 0],
#         features_2d[poison_idx, 1],
#         marker='^',       # 三角形
#         facecolors='none', 
#         edgecolors='red',
#         s=140,
#         alpha=0.8,
#         linewidth=1.5,
#         label='Poisoned'
#     )

# plt.title("Feature Representation")
# # plt.xlabel("Principal Component 1")
# # plt.ylabel("Principal Component 2")
# # plt.xlabel("Principal Component 1")
# # plt.ylabel("Principal Component 2")
# plt.grid(True, linestyle='--', alpha=0.3)
# plt.legend(loc='best')
# plt.tight_layout()
# plt.savefig("pca_subsampled_0.1-2.png",dpi=1200)
# plt.show()

/tmp/ipykernel_3637823/2317558040.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap("tab20b", len(unique_labels))


In [18]:
# import pandas as pd
# import numpy as np

# # 1. 获取防御模型的预测结果
# # final_defence 为经过激活聚类检测后得到的防御器实例，其 is_clean_lst 属性保存预测结果（1 表示干净，0 表示投毒）
# is_clean_predicted = np.array(final_defence.is_clean_lst)

# # 2. 保持原始数据结构，并添加预测标记
# n_samples = len(all_features)
# # 确保 all_features 和 all_activations 为二维数组（如果本身已经是二维则可直接使用）
# features_2d = all_features.reshape(n_samples, -1)
# activations_2d = all_activations.reshape(n_samples, -1)

# # 创建 DataFrame，包含原始特征和激活信息
# df_features = pd.DataFrame(features_2d, columns=[f'feature_{i}' for i in range(features_2d.shape[1])])
# df_activations = pd.DataFrame(activations_2d, columns=[f'activation_{i}' for i in range(activations_2d.shape[1])])
# df = pd.concat([df_features, df_activations], axis=1)
# # 添加原始标签和投毒标记
# df['label'] = all_labels
# df['is_poisoned'] = all_is_poisoned
# df['is_clean_predicted'] = is_clean_predicted  # 添加防御模型的预测结果

In [19]:
# import os
# import sys
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, row_number, rand, monotonically_increasing_id,input_file_name, regexp_extract
# from pyspark.sql.window import Window
# import psutil
# from pyspark.sql.types import StructType, StructField, ArrayType, DoubleType, LongType, BooleanType

# output_dir="cleaned_data"
# # 3. 转换为 Spark DataFrame 并保存
# # 假设 spark 为已初始化的 SparkSession，save_test 为保存函数，output_dir 为输出目录
# os.environ["PYSPARK_PYTHON"] = sys.executable
# os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
# memory_gb = psutil.virtual_memory().available // 1024 // 1024 // 1024
# print("memory_gb:",memory_gb)
# # 开始计时
# start_time = time.time()
# spark = (
#     SparkSession.builder.master("local[*]") 
#     .config("spark.driver.memory", "50g")

#     .config("spark.sql.adaptive.enabled", "true")# 启用自适应查询执行
#     .config("spark.sql.adaptive.coalescePartitions.enabled", "true")# 启用分区合并

#     .config("spark.executor.memory", "16g")  # 每个Executor 48GB内存
#     .config("spark.executor.cores", "2")  # 每个Executor 8个核心

#     .config("spark.memory.fraction", "0.6")          
#     .config("spark.memory.storageFraction", "0.4")  

#     .config("spark.driver.host", "127.0.0.1")
#     # .config("spark.sql.files.ignoreCorruptFiles", "true")  # 跳过损坏文件
#     # .config("spark.sql.files.ignoreMissingFiles", "true") # 跳过缺失文件
#     .config("spark.dynamicAllocation.enabled", "false")
#     .config("spark.executor.instances", "1")        # 固定4个Executor

#     .config("spark.dynamicAllocation.executorIdleTimeout", "60s")  # 设置 Executor 空闲超时时间
#     .config("spark.dynamicAllocation.cachedExecutorIdleTimeout", "60s")  # 设置缓存的 Executor 空闲超时时间
#     .config("spark.sql.shuffle.partitions", "100") # 每核心约4~6个分区

#     .config("spark.cleaner.ttl", 7200) # 清理时间间隔为1小时
#     .config("spark.ui.port", "4041")
#     .config("spark.local.dir","/data/zmx/tmp")  # Spark临时文件目录 ,"/home/zmx/tmp"
#     .getOrCreate()
# )
# spark_df = spark.createDataFrame(df)
# save_test(spark_df, output_dir)

memory_gb: 126


25/05/11 17:14:39 WARN Utils: Your hostname, hipeson resolves to a loopback address: 127.0.1.1; using 172.23.216.92 instead (on interface eno2np1)
25/05/11 17:14:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 17:14:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/11 17:14:40 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [ ]:

# 4. 输出数据统计信息
print("数据统计：")
print(f"总样本数: {len(df)}")
print(f"预测为干净的样本数: {np.sum(is_clean_predicted == 1)}")
print(f"预测为中毒的样本数: {np.sum(is_clean_predicted == 0)}")